In [8]:
from readers import lenta_reader, ria_reader
import tqdm
from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from transformers import AutoTokenizer, EncoderDecoderModel, EncoderDecoderConfig, \
    BertConfig, PreTrainedModel, PretrainedConfig, \
    Trainer, TrainingArguments, logging

import pandas as pd
import csv
import random

In [2]:
lenta_path = '/home/aobuhtijarov/datasets/lenta/lenta-ru-news.train.csv'
ria_path = '/home/aobuhtijarov/datasets/ria/ria.shuffled.train.json'

In [4]:
lenta_records = [r for r in tqdm.tqdm(lenta_reader(lenta_path))]
ria_records = [r for r in tqdm.tqdm(ria_reader(ria_path))]


597175it [00:20, 29709.94it/s]
891612it [10:34, 1404.58it/s]


In [14]:
val_lenta_records = [r for r in tqdm.tqdm(lenta_reader('/home/aobuhtijarov/datasets/lenta/lenta-ru-news.val.csv'))]

75971it [00:02, 36925.84it/s]


75971

In [15]:
sum(1 for r in val_lenta_records if r['date'][:4] in ['2010', '2011', '2012', '2013', '2014'])

21376

In [5]:
len(lenta_records), len(ria_records)

(597175, 891612)

In [9]:
random.shuffle(ria_records)

In [12]:
val_ria_records = [r for r in tqdm.tqdm(ria_reader('/home/aobuhtijarov/datasets/ria/ria.shuffled.val.json'))]

49261it [00:27, 1792.26it/s]


In [13]:
len(val_ria_records)

49261

In [21]:
x = [2,1]
x.extend([1, 5])
x

[2, 1, 1, 5]

In [3]:
cd ../../datasets/

/home/aobuhtijarov/datasets


In [19]:
400000 * 0.05

20000.0

In [2]:
logging.set_verbosity_info()

In [3]:
import json
import random
import copy
import torch
import tqdm

In [4]:
from _jsonnet import evaluate_file as jsonnet_evaluate_file

In [5]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

tokenizer_model_path = '/Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False)

loading configuration file /Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

Model name '/Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-

In [6]:
eval_model_file = '/Users/leshanbog/Documents/neural_models/checkpoint-7000/'
enable_bottleneck = True

In [7]:
test_records = [r for r in ria_reader('/Users/leshanbog/Documents/dataset/ria/ria.shuffled.test.json') if random.random() <= 0.005]

In [8]:
test_dataset = GenTitleDataset(
    test_records,
    tokenizer,
    max_tokens_text=196,
    max_tokens_title=48
)

In [9]:
cls = BottleneckEncoderDecoderModel if enable_bottleneck else EncoderDecoderModel
model = cls.from_pretrained(eval_model_file)
model.eval()

loading configuration file /Users/leshanbog/Documents/neural_models/checkpoint-7000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "/data/aobuhtijarov/models/pretrained_dec_6_layers",
    "add_cross_attention": true,
    "architectures": [
      "BertModel"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "eos_token_id": null,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "lab

BottleneckEncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [10]:
batch_size = 6

In [11]:
for i in tqdm.trange(0, len(test_dataset), batch_size):
    data = test_dataset[i]
    for k in tuple(data.keys()):
        if k not in ('input_ids', 'attention_mask'):
            del data[k]
        else:
            data[k] = data[k].unsqueeze(0)

    for j in range(i + 1, min(i + batch_size, len(test_dataset))):
        for k in data.keys():
            data[k] = torch.cat((data[k], test_dataset[j][k].unsqueeze(0)), dim=0)


    output_ids = model.generate(**data, decoder_start_token_id=model.config.decoder.pad_token_id)
    preds = [
        tokenizer.decode(x[1: torch.max(torch.nonzero(x)).item()]) for x in output_ids
    ]
    
    break

  0%|          | 0/40 [00:01<?, ?it/s]


ValueError: You have to specify either input_ids or inputs_embeds

In [26]:
txt = 'как дела у тебя дружочек братишка?'

inp2 = tokenizer(
            txt,
            add_special_tokens=True,
            max_length=196,
            padding="max_length",
            truncation=True
        )

inp = dict()
inp['input_ids'] = torch.LongTensor(inp2['input_ids']).unsqueeze(0)
inp['attention_mask'] = torch.LongTensor(inp2['attention_mask']).unsqueeze(0)

In [27]:
model.generate(**inp, decoder_start_token_id=model.config.decoder.pad_token_id)

ValueError: You have to specify either input_ids or inputs_embeds

In [1]:
from sklearn.cluster import AgglomerativeClustering
from _jsonnet import evaluate_file as jsonnet_evaluate_file
import json
from transformers import EncoderDecoderModel, logging, BertTokenizer

from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from utils.clustering_utils import calc_clustering_metrics, get_text_to_vector_func

logging.set_verbosity_info()

In [2]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

max_tokens_text = config.pop('max_tokens_text')
max_tokens_title = config.pop('max_tokens_title')
text_to_vec_func = 'bert-MeanSum'

In [3]:
model_path = '/home/aobuhtijarov/models/tg_gen_title_1500/checkpoint-6000/'
gold_markup_file = '/home/aobuhtijarov/datasets/telegram_news/ru_clustering_0517.tsv'
clustering_data_file = '/home/aobuhtijarov/datasets/telegram_news/ru_clustering_data.jsonl'
tokenizer_model_path = '/home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt'

In [4]:
model = EncoderDecoderModel.from_pretrained(model_path)
model.eval()
model.cuda()

loading configuration file /home/aobuhtijarov/models/tg_gen_title_1500/checkpoint-6000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "/home/aobuhtijarov/models/pretrained_dec_6_layers",
    "add_cross_attention": true,
    "architectures": [
      "BertModel"
    ],
    "attention_probs_dropout_prob": 0.2,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.2,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_ra

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [5]:
from utils.clustering_utils import *
import numpy as np

In [6]:
tokenizer = BertTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False, do_basic_tokenize=False)

Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/added_tokens.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/special_tokens_map.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer_config.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer.json. We won't load it.
loading file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/vocab.txt
loading file None
loading file None
loading file None
loading file None


In [7]:
gold_markup = get_gold_markup(gold_markup_file)

url2record, filename2url = get_data_to_cluster(clustering_data_file)
setattr(tokenizer, 'max_tokens_text', max_tokens_text)
text_to_vector_func = get_text_to_vector_func(text_to_vec_func, model, tokenizer)

print('Calculating embeddings...')
embeds = np.zeros((len(url2record.items()), 768))

Calculating embeddings...


In [8]:
import json
import tqdm
from sklearn.cluster import AgglomerativeClustering

In [9]:
total_articles = len(url2record.items())

for i, (url, record) in tqdm.tqdm(enumerate(url2record.items()), total=total_articles):
    text = record["title"] + ' ' + record["text"]
    text = text.lower().replace('\xa0', ' ').strip()
    embeds[i] = text_to_vector_func(text).detach().cpu().numpy().ravel()

100%|██████████| 10730/10730 [02:49<00:00, 63.41it/s]


In [ ]:
clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=0.13,
        linkage="single",
        affinity="cosine"
    )

clustering_model.fit(embeds)
labels = clustering_model.labels_

In [ ]:
id2url = dict()
for i, (url, _) in enumerate(url2record.items()):
    id2url[i] = url

url2label = dict()
for i, label in enumerate(labels):
    url2label[id2url[i]] = label

In [ ]:
not_found_count = 0
for first_url, second_url in list(gold_markup.keys()):
    not_found_in_labels = first_url not in url2label or second_url not in url2label
    not_found_in_records = first_url not in url2record or second_url not in url2record
    if not_found_in_labels or not_found_in_records:
        not_found_count += 1
        print(first_url, second_url)
        gold_markup.pop((first_url, second_url))

In [ ]:
not_found_count